In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

from sklearn.preprocessing import MultiLabelBinarizer

from warnings import simplefilter
simplefilter("ignore")

In [2]:
data = pd.read_csv('all_restaurants.csv')
data

,name,address,coordinates,rating,reviews,categories,avg_bill,cuisine,assortment
0,"Паруса на крыше, панорамный ресторан","улица Льва Толстого, 9","{'lat': 59.965318, 'lon': 30.31563}",4.6,2503,"['Рестораны', 'Банкетные залы', 'Караоке-залы'...",Средний чек 1500 ₽,"['Японская кухня', 'Русская кухня', 'Авторская...","['Бургеры', 'Пельмени', 'Рамен', 'Стейки', 'То..."
1,"Doski, бар-ресторан","Звенигородская улица, 1 к2","{'lat': 59.922348, 'lon': 30.335726}",4.0,2220,"['Рестораны', 'Ночные клубы', 'Бары', 'Караоке...",Средний чек 600 ₽,[],"['Суши', 'Пицца', 'Салаты', 'Супы', 'Куриные к..."
2,"Евразия, рестораны японской и европейской кухни","Невский проспект, 42","{'lat': 59.934812, 'lon': 30.332519}",4.8,2863,"['Рестораны', 'Суши-бары', 'Доставка еды', 'Ба...",Средний чек 1500 ₽,"['Азиатская кухня', 'Европейская кухня']","['Суши', 'Лапша wok', 'Пельмени', 'Поке', 'Сте..."
3,"Art Lab Medovarus, фирменный бар","Мучной переулок, 5","{'lat': 59.930072, 'lon': 30.322454}",4.8,290,"['Рестораны', 'Бары', 'Кафе']",Средний чек 1100 ₽,[],"['Стейки', 'Салаты', 'Супы', 'Начос']"
4,"The бык, демократичный мясной ресторан","Невский проспект, 55","{'lat': 59.932201, 'lon': 30.350266}",4.5,1231,['Рестораны'],Средний чек 700 ₽,['Европейская кухня'],"['Пицца', 'Шашлык', 'Бургеры', 'Стейки', 'Том-..."
...,...,...,...,...,...,...,...,...,...
295,"Rostic`s, ресторан быстрого обслуживания","Загородный проспект, 2","{'lat': 59.927773, 'lon': 30.346864}",2.4,340,"['Быстрое питание', 'Доставка еды']",NaN,[],[]
296,"Easy sushi, суши-ресторан","Садовая улица, 33","{'lat': 59.928346, 'lon': 30.321154}",3.7,309,"['Быстрое питание', 'Суши-бары', 'Доставка еды']",Средний чек 550 ₽,['Японская кухня'],"['Суши', 'Лапша wok', 'Рис wok', 'Онигири']"
297,"Ramen Slurp Bar, раменная","Жуковского, 31","{'lat': 59.935771, 'lon': 30.356185}",3.9,260,"['Быстрое питание', 'Доставка еды']",Средний чек 400 ₽,"['Японская кухня', 'Паназиатская кухня']","['Лапша wok', 'Рамен', 'Рис wok', 'Салаты', 'С..."
298,"Babaganoush, кафе израильской кухни","Восстания, 19","{'lat': 59.93515, 'lon': 30.360435}",4.5,163,"['Быстрое питание', 'Доставка еды']",Средний чек 950 ₽,"['Восточная кухня', 'Израильская кухня']","['Шаурма', 'Фалафель', 'Картофель-фри', 'Салат..."


In [3]:
before = data.shape[0]
after = data.drop_duplicates().shape[0]
print(f'Удалено дубликатов : {(before - after)}')

Удалено дубликатов : 1


In [4]:
len(data['coordinates'].unique())

242

##### Оставляем фичи для ранжирования 

In [5]:
imp_features = ['name', 'rating', 'reviews', 'avg_bill', 'cuisine']

data_imp = data[imp_features].copy(deep=True)
data_imp

,name,rating,reviews,avg_bill,cuisine
0,"Паруса на крыше, панорамный ресторан",4.6,2503,Средний чек 1500 ₽,"['Японская кухня', 'Русская кухня', 'Авторская..."
1,"Doski, бар-ресторан",4.0,2220,Средний чек 600 ₽,[]
2,"Евразия, рестораны японской и европейской кухни",4.8,2863,Средний чек 1500 ₽,"['Азиатская кухня', 'Европейская кухня']"
3,"Art Lab Medovarus, фирменный бар",4.8,290,Средний чек 1100 ₽,[]
4,"The бык, демократичный мясной ресторан",4.5,1231,Средний чек 700 ₽,['Европейская кухня']
...,...,...,...,...,...
295,"Rostic`s, ресторан быстрого обслуживания",2.4,340,NaN,[]
296,"Easy sushi, суши-ресторан",3.7,309,Средний чек 550 ₽,['Японская кухня']
297,"Ramen Slurp Bar, раменная",3.9,260,Средний чек 400 ₽,"['Японская кухня', 'Паназиатская кухня']"
298,"Babaganoush, кафе израильской кухни",4.5,163,Средний чек 950 ₽,"['Восточная кухня', 'Израильская кухня']"


##### Удаляем пропущенные значения

In [6]:
data_imp = data_imp.replace('[]', np.nan)
data_imp

,name,rating,reviews,avg_bill,cuisine
0,"Паруса на крыше, панорамный ресторан",4.6,2503,Средний чек 1500 ₽,"['Японская кухня', 'Русская кухня', 'Авторская..."
1,"Doski, бар-ресторан",4.0,2220,Средний чек 600 ₽,NaN
2,"Евразия, рестораны японской и европейской кухни",4.8,2863,Средний чек 1500 ₽,"['Азиатская кухня', 'Европейская кухня']"
3,"Art Lab Medovarus, фирменный бар",4.8,290,Средний чек 1100 ₽,NaN
4,"The бык, демократичный мясной ресторан",4.5,1231,Средний чек 700 ₽,['Европейская кухня']
...,...,...,...,...,...
295,"Rostic`s, ресторан быстрого обслуживания",2.4,340,NaN,NaN
296,"Easy sushi, суши-ресторан",3.7,309,Средний чек 550 ₽,['Японская кухня']
297,"Ramen Slurp Bar, раменная",3.9,260,Средний чек 400 ₽,"['Японская кухня', 'Паназиатская кухня']"
298,"Babaganoush, кафе израильской кухни",4.5,163,Средний чек 950 ₽,"['Восточная кухня', 'Израильская кухня']"


In [7]:
data_imp_filled = data_imp.dropna()
data_imp_filled

,name,rating,reviews,avg_bill,cuisine
0,"Паруса на крыше, панорамный ресторан",4.6,2503,Средний чек 1500 ₽,"['Японская кухня', 'Русская кухня', 'Авторская..."
2,"Евразия, рестораны японской и европейской кухни",4.8,2863,Средний чек 1500 ₽,"['Азиатская кухня', 'Европейская кухня']"
4,"The бык, демократичный мясной ресторан",4.5,1231,Средний чек 700 ₽,['Европейская кухня']
5,"Большой Грузинский, ресторан",4.7,1205,Средний чек 1300 ₽,"['Грузинская кухня', 'Кавказская кухня']"
6,"Feromon Group, лаундж-бар",4.9,726,Средний чек 1200 ₽,"['Итальянская кухня', 'Паназиатская кухня', 'Е..."
...,...,...,...,...,...
292,"Теремок, ресторан домашней кухни",4.2,305,Средний чек 350 ₽,['Русская кухня']
294,"Лео, шаверма-бар",4.6,990,Средний чек 1000 ₽,['Восточная кухня']
296,"Easy sushi, суши-ресторан",3.7,309,Средний чек 550 ₽,['Японская кухня']
297,"Ramen Slurp Bar, раменная",3.9,260,Средний чек 400 ₽,"['Японская кухня', 'Паназиатская кухня']"


##### Выводим средний чек

In [8]:
def extract_number(text):
    # Ищем число в строке с помощью регулярного выражения
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())  # Возвращаем найденное число
    return None  # Если число не найдено, возвращаем None

In [9]:
data_imp_filled['avg_bill'] = data_imp_filled['avg_bill'].apply(extract_number)

data_imp_filled

,name,rating,reviews,avg_bill,cuisine
0,"Паруса на крыше, панорамный ресторан",4.6,2503,1500,"['Японская кухня', 'Русская кухня', 'Авторская..."
2,"Евразия, рестораны японской и европейской кухни",4.8,2863,1500,"['Азиатская кухня', 'Европейская кухня']"
4,"The бык, демократичный мясной ресторан",4.5,1231,700,['Европейская кухня']
5,"Большой Грузинский, ресторан",4.7,1205,1300,"['Грузинская кухня', 'Кавказская кухня']"
6,"Feromon Group, лаундж-бар",4.9,726,1200,"['Итальянская кухня', 'Паназиатская кухня', 'Е..."
...,...,...,...,...,...
292,"Теремок, ресторан домашней кухни",4.2,305,350,['Русская кухня']
294,"Лео, шаверма-бар",4.6,990,1000,['Восточная кухня']
296,"Easy sushi, суши-ресторан",3.7,309,550,['Японская кухня']
297,"Ramen Slurp Bar, раменная",3.9,260,400,"['Японская кухня', 'Паназиатская кухня']"


##### Кодируем кухни

In [10]:
#pip install ast

  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [8 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Максим\AppData\Local\Temp\pip-install-qy7gz8kh\ast_a7fddc1eae4b4e03be02bb2da0c04ccc\setup.py", line 6, in <module>
      README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "<frozen codecs>", line 918, in open
  FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Максим\\AppData\\Local\\Temp\\pip-install-qy7gz8kh\\ast_a7fddc1eae4b4e03be02bb2da0c04ccc\\AST/README'
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for outpu

In [12]:
# Функция для преобразования строки в список и извлечения кухонь
def extract_cuisines(cuisine_str):
    try:
        # Преобразуем строку в список
        cuisine_list = ast.literal_eval(cuisine_str)
        # Объединяем элементы списка в строку через запятую
        return ', '.join(cuisine_list)
    except (ValueError, SyntaxError):
        # Если преобразование не удалось, возвращаем исходное значение
        return cuisine_str

# Применяем функцию к столбцу
data_imp_filled['cuisine'] = data_imp_filled['cuisine'].apply(extract_cuisines)
data_imp_filled

,name,rating,reviews,avg_bill,cuisine
0,"Паруса на крыше, панорамный ресторан",4.6,2503,1500,"Японская кухня, Русская кухня, Авторская кухня"
2,"Евразия, рестораны японской и европейской кухни",4.8,2863,1500,"Азиатская кухня, Европейская кухня"
4,"The бык, демократичный мясной ресторан",4.5,1231,700,Европейская кухня
5,"Большой Грузинский, ресторан",4.7,1205,1300,"Грузинская кухня, Кавказская кухня"
6,"Feromon Group, лаундж-бар",4.9,726,1200,"Итальянская кухня, Паназиатская кухня, Европей..."
...,...,...,...,...,...
292,"Теремок, ресторан домашней кухни",4.2,305,350,Русская кухня
294,"Лео, шаверма-бар",4.6,990,1000,Восточная кухня
296,"Easy sushi, суши-ресторан",3.7,309,550,Японская кухня
297,"Ramen Slurp Bar, раменная",3.9,260,400,"Японская кухня, Паназиатская кухня"


In [19]:
#from sklearn.preprocessing import MultiLabelBinarizer

In [34]:
# Шаг 1: Собираем все уникальные значения кухонь
unique_cuisines = set(cuisine for sublist in data_imp_filled['cuisine'] for cuisine in sublist)

# Преобразуем множество в отсортированный список
unique_cuisines = sorted(unique_cuisines)

# Выводим результат
print(f"Число уникальных кухонь: {len(unique_cuisines)}")
print(unique_cuisines)

Число уникальных кухонь: 22
['Авторская кухня', 'Азиатская кухня', 'Американская кухня', 'Веганская кухня', 'Восточная кухня', 'Грузинская кухня', 'Европейская кухня', 'Израильская кухня', 'Испанская кухня', 'Итальянская кухня', 'Кавказская кухня', 'Китайская кухня', 'Корейская кухня', 'Мексиканская кухня', 'Немецкая кухня', 'Паназиатская кухня', 'Русская кухня', 'Средиземноморская кухня', 'Тайская кухня', 'Узбекская кухня', 'Французская кухня', 'Японская кухня']


In [39]:
for index, row in data_imp_filled.iterrows():
    for cuisine in row['cuisine']:
        if cuisine in unique_cuisines:
            data_imp_filled.at[index, cuisine] = 1

data_imp_filled.drop('cuisine', axis=1, inplace=True)

In [40]:
data_imp_filled

,name,rating,reviews,avg_bill,Авторская кухня,Азиатская кухня,Американская кухня,Веганская кухня,Восточная кухня,Грузинская кухня,...,Корейская кухня,Мексиканская кухня,Немецкая кухня,Паназиатская кухня,Русская кухня,Средиземноморская кухня,Тайская кухня,Узбекская кухня,Французская кухня,Японская кухня
0,"Паруса на крыше, панорамный ресторан",4.6,2503,1500,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,"Евразия, рестораны японской и европейской кухни",4.8,2863,1500,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The бык, демократичный мясной ресторан",4.5,1231,700,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Большой Грузинский, ресторан",4.7,1205,1300,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,"Feromon Group, лаундж-бар",4.9,726,1200,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,"Теремок, ресторан домашней кухни",4.2,305,350,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
294,"Лео, шаверма-бар",4.6,990,1000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
296,"Easy sushi, суши-ресторан",3.7,309,550,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
297,"Ramen Slurp Bar, раменная",3.9,260,400,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
